In [34]:
import pandas as pd
covid_data = pd.read_csv("datasets\covid-data.csv")

In [51]:
poland_covid_data=covid_data[covid_data['location']=='Poland']

In [36]:
poland_covid_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 631 entries, 97930 to 98560
Data columns (total 67 columns):
 #   Column                                      Non-Null Count  Dtype  
---  ------                                      --------------  -----  
 0   iso_code                                    631 non-null    object 
 1   continent                                   631 non-null    object 
 2   location                                    631 non-null    object 
 3   date                                        631 non-null    object 
 4   total_cases                                 631 non-null    float64
 5   new_cases                                   631 non-null    float64
 6   new_cases_smoothed                          626 non-null    float64
 7   total_deaths                                623 non-null    float64
 8   new_deaths                                  623 non-null    float64
 9   new_deaths_smoothed                         626 non-null    float64
 10  total_ca

In [43]:
poland_covid_data.describe().to_excel('describe.xlsx')

In [53]:
pol_cov_daily=poland_covid_data.drop(columns=['iso_code', 'continent', 'location', 'tests_units', 'human_development_index', 'life_expectancy', 'hospital_beds_per_thousand', 'handwashing_facilities', 'male_smokers', 'female_smokers', 'diabetes_prevalence', 'cardiovasc_death_rate', 'extreme_poverty', 'gdp_per_capita', 'aged_70_older', 'aged_65_older', 'median_age', 'population_density', 'population', 'new_people_vaccinated_smoothed_per_hundred', 'new_people_vaccinated_smoothed', 'new_vaccinations_smoothed_per_million', 'total_boosters_per_hundred', 'people_fully_vaccinated_per_hundred', 'people_vaccinated_per_hundred', 'total_vaccinations_per_hundred', 'new_vaccinations_smoothed', 'new_vaccinations', 'total_boosters', 'people_fully_vaccinated', 'people_vaccinated', 'total_vaccinations', 'positive_rate', 'new_tests_smoothed_per_thousand', 'new_tests_smoothed', 'new_tests_per_thousand', 'total_tests_per_thousand', 'total_tests', 'weekly_hosp_admissions_per_million', 'weekly_hosp_admissions', 'weekly_icu_admissions_per_million', 'weekly_icu_admissions', 'hosp_patients_per_million', 'icu_patients_per_million', 'icu_patients', 'reproduction_rate', 'new_deaths_smoothed_per_million', 'new_deaths_per_million', 'total_deaths_per_million', 'new_cases_smoothed_per_million', 'new_cases_per_million', 'total_cases_per_million', 'new_deaths_smoothed', 'total_deaths', 'new_cases_smoothed', 'total_cases', 'excess_mortality_cumulative_per_million'])
pol_cov_daily

,date,new_cases,new_deaths,hosp_patients,new_tests,tests_per_case,stringency_index,excess_mortality_cumulative_absolute,excess_mortality_cumulative,excess_mortality
97930,2020-03-04,1.0,NaN,NaN,NaN,NaN,11.11,NaN,NaN,NaN
97931,2020-03-05,0.0,NaN,NaN,NaN,NaN,11.11,NaN,NaN,NaN
97932,2020-03-06,4.0,NaN,NaN,NaN,NaN,11.11,NaN,NaN,NaN
97933,2020-03-07,0.0,NaN,NaN,NaN,NaN,11.11,NaN,NaN,NaN
97934,2020-03-08,6.0,NaN,NaN,NaN,NaN,11.11,-6305.6,-7.01,-6.3
...,...,...,...,...,...,...,...,...,...,...
98556,2021-11-20,23418.0,382.0,NaN,95486.0,4.1,41.67,NaN,NaN,NaN
98557,2021-11-21,18924.0,41.0,NaN,67381.0,4.1,41.67,NaN,NaN,NaN
98558,2021-11-22,12375.0,8.0,NaN,48557.0,4.1,41.67,NaN,NaN,NaN
98559,2021-11-23,19935.0,398.0,NaN,NaN,NaN,41.67,NaN,NaN,NaN


In [49]:
value_columns=list(set(pol_cov_daily.columns)-{'date'})

In [50]:
import numpy as np
from sklearn.impute import KNNImputer

imputer = KNNImputer(n_neighbors=5)
pol_cov_daily[value_columns] = imputer.fit_transform(pol_cov_daily[value_columns])

In [57]:
def z_score(series : pd.Series):
    mean = np.mean(series)
    std = np.std(series)
    return (series-mean)/std

outlier_index = abs(z_score(pol_cov_daily['new_cases']))>3
pol_cov_daily['new_cases'][outlier_index]

98195    32733.0
98315    30802.0
98316    34150.0
98317    35145.0
98318    31759.0
98319    29266.0
98322    32891.0
98323    35253.0
98324    30541.0
Name: new_cases, dtype: float64

In [58]:
from scipy import stats

def hampel(series : pd.Series):
    median = np.median(series)
    mad = stats.median_absolute_deviation(series, scale=1)
    return (series-median)/mad

outlier_index = abs(hampel(pol_cov_daily['new_cases']))>3
pol_cov_daily['new_cases'][outlier_index]

<ipython-input-58-7b42e3bece84>:5: DeprecationWarning: `median_absolute_deviation` is deprecated, use `median_abs_deviation` instead!

To preserve the existing default behavior, use
`scipy.stats.median_abs_deviation(..., scale=1/1.4826)`.
The value 1.4826 is not numerically precise for scaling
with a normal distribution. For a numerically precise value, use
`scipy.stats.median_abs_deviation(..., scale='normal')`.

  mad = stats.median_absolute_deviation(series, scale=1)


98147     3003.0
98148     4280.0
98149     4739.0
98150     5300.0
98151     4178.0
          ...   
98556    23418.0
98557    18924.0
98558    12375.0
98559    19935.0
98560    28431.0
Name: new_cases, Length: 251, dtype: float64